In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 35.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/data/
%ls

/content/drive/MyDrive/data
compare_5/                 gpt2_predict_model/
finalWords.json            gpt2_predict_model_30/
finetune_eval.txt          gpt2_predict_model_6/
finetune_train.txt         predict_embeddings_model_weights
finetune_val.txt           predict_embeddings_model_weights_30
gpt2_adapter_casual/       predict_embeddings_model_weights_6
gpt2_adapter_mlm_model/    quiz_questions_10.txt
gpt2_adapter_mlm_model_3/  quiz_questions_25.txt
gpt2_casual/               quiz_questions_2.txt
gpt2_casual_2              quiz_questions_50.txt
gpt2_finetune_random/      quiz_questions_5.txt
gpt2_mlm_model/            quiz_questions.txt
gpt2_mlm_model_2/          words.txt
gpt2_mlm_model_3/


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelWithLMHead
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import json
import random
from transformers import AutoTokenizer, GPT2DoubleHeadsModel

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
# Load the JSON file
# Load the JSON file
with open('finalWords.json') as f:
    words = json.load(f)

model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = GPT2DoubleHeadsModel.from_pretrained("gpt2")
model = model.to(device)

original_vocab_size = len(tokenizer)
new_word_list = [i.strip().lower() for i in open("words.txt").readlines()]
num_new_words = len(new_word_list)
num_added = tokenizer.add_tokens(new_word_list)
model.resize_token_embeddings(len(tokenizer))

#Compute the distribution from which we’ll sample:
params = model.state_dict()
embeddings = params['transformer.wte.weight']
pre_expansion_embeddings = embeddings[:-num_new_words,:]
mu = torch.mean(pre_expansion_embeddings, dim=0)
n = pre_expansion_embeddings.size()[0]
sigma = ((pre_expansion_embeddings - mu).T @ (pre_expansion_embeddings - mu)) / n
dist = torch.distributions.multivariate_normal.MultivariateNormal(
        mu, covariance_matrix=1e-5*sigma)

#load in our new embeddings into the model:
new_embeddings = torch.stack(tuple((dist.sample() for _ in range(num_new_words))), dim=0)
embeddings[-num_new_words:,:] = new_embeddings
params['transformer.wte.weight'][-num_new_words:,:] = new_embeddings
model.load_state_dict(params)

Some weights of GPT2DoubleHeadsModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['multiple_choice_head.summary.bias', 'multiple_choice_head.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [ ]:
# print(new_word_list)
print(len(new_word_list))

50001


In [ ]:
import json
with open("quiz_questions.txt") as f:
  quiz_questions = json.load(f)
print(len(quiz_questions))

12211


In [ ]:
import json

num_added_tokens = tokenizer.add_special_tokens({"cls_token": "[CLS]"})
# Update the model embeddings with the new vocabulary size
# print("tokenizer length", len(tokenizer))
embedding_layer = model.resize_token_embeddings(len(tokenizer))

num_choices_arr = [5]
model_ans = []
for num_choices in num_choices_arr:

  with open(f"quiz_questions_{num_choices}.txt") as f:
    quiz_questions = json.load(f)
  print("NUM_CHOICES:", num_choices)

 

  questions = quiz_questions
  # answers = [0 for i in range(0, questions)]

  counts = [0.0 for i in range(0, num_choices)]
  total = 0.0
  # print("question", len(questions))
  model.eval()
  with torch.no_grad():        
    for i, choices in enumerate(questions):
        #print(choices)
        # print(i)
        if i % 1000 == 0 and i != 0:
          print(f"On iteration {i}, accuracy is {counts[0]/total} equal would be {1/num_choices} count is {total}", [i/total for i in counts])
        encoded_choices = [tokenizer.encode(s) for s in choices]
        cls_token_location = [tokens.index(tokenizer.cls_token_id) for tokens in encoded_choices]
        #check if some choices are longer than the others
        max_len = max([len(tokens) for tokens in encoded_choices])
        check = True
        for tokens in encoded_choices:
            if len(tokens) < max_len:
                check = False
                break
        if check:
            input_ids = torch.tensor(encoded_choices).unsqueeze(0)  # Batch size: 1, number of choices: 2
            if len(input_ids) >= 1024:
              continue
            mc_token_ids = torch.tensor([cls_token_location])  # Batch size: 1
            input_ids = input_ids.to(device)
            mc_token_ids = mc_token_ids.to(device)
            outputs = model(input_ids, mc_token_ids=mc_token_ids)
            lm_logits = outputs.logits
            mc_logits = outputs.mc_logits
            counts[mc_logits.argmax().item()] += 1
            total += 1
            model_ans.append(mc_logits.argmax().item())
        else:
          model_ans.append(-1)
        torch.cuda.empty_cache() 
  print(f"FINAL for num_choices {num_choices}: accuracy is {counts[0]/total} equal would be {1/num_choices} count is {total}", [i/total for i in counts])

  # print(counts)
  # print(total)

NUM_CHOICES: 5
On iteration 1000, accuracy is 0.18856569709127383 equal would be 0.2 count is 997.0 [0.18856569709127383, 0.20160481444332998, 0.19658976930792377, 0.21063189568706117, 0.20260782347041123]
On iteration 2000, accuracy is 0.1844758064516129 equal would be 0.2 count is 1984.0 [0.1844758064516129, 0.19556451612903225, 0.20413306451612903, 0.19808467741935484, 0.21774193548387097]
On iteration 3000, accuracy is 0.1850235057085292 equal would be 0.2 count is 2978.0 [0.1850235057085292, 0.19476158495634655, 0.20752182672934855, 0.19811954331766285, 0.21457353928811282]
On iteration 4000, accuracy is 0.18816120906801007 equal would be 0.2 count is 3970.0 [0.18816120906801007, 0.20503778337531486, 0.2040302267002519, 0.1929471032745592, 0.209823677581864]
On iteration 5000, accuracy is 0.1928657799274486 equal would be 0.2 count is 4962.0 [0.1928657799274486, 0.1985086658605401, 0.20354695687222893, 0.196291817815397, 0.20878677952438532]
On iteration 6000, accuracy is 0.195134

In [ ]:
f = open("compare_5/average_baseline.txt", "w+")
for i in model_ans:
  f.write(str(i)+"\n")
f.close()
print(len(model_ans))

12211


In [ ]:
import numpy as np
print(data)
print("mean", np.mean(data))
print("std", np.std(data))

[0.19643972517176764, 0.16958151155527795, 0.16895690193628982, 0.18332292317301688, 0.21018113678950656, 0.17051842598376016, 0.16833229231730168, 0.1867582760774516, 0.17395377888819488, 0.1858213616489694]
mean 0.18138663335415364
std 0.013212228354396763


In [ ]:
def trial():
  model_name = 'gpt2'
  tokenizer = AutoTokenizer.from_pretrained("gpt2")
  model = GPT2DoubleHeadsModel.from_pretrained("gpt2")
  model = model.to(device)

  original_vocab_size = len(tokenizer)
  new_word_list = [i.strip().lower() for i in open("words.txt").readlines()]
  num_new_words = len(new_word_list)
  num_added = tokenizer.add_tokens(new_word_list)
  model.resize_token_embeddings(len(tokenizer))

  #Compute the distribution from which we’ll sample:
  params = model.state_dict()
  embeddings = params['transformer.wte.weight']
  pre_expansion_embeddings = embeddings[:-num_new_words,:]
  mu = torch.mean(pre_expansion_embeddings, dim=0)
  n = pre_expansion_embeddings.size()[0]
  sigma = ((pre_expansion_embeddings - mu).T @ (pre_expansion_embeddings - mu)) / n
  dist = torch.distributions.multivariate_normal.MultivariateNormal(
          mu, covariance_matrix=1e-5*sigma)

  #load in our new embeddings into the model:
  new_embeddings = torch.stack(tuple((dist.sample() for _ in range(num_new_words))), dim=0)
  embeddings[-num_new_words:,:] = new_embeddings
  params['transformer.wte.weight'][-num_new_words:,:] = new_embeddings
  model.load_state_dict(params)



  num_added_tokens = tokenizer.add_special_tokens({"cls_token": "[CLS]"})
  # Update the model embeddings with the new vocabulary size
  embedding_layer = model.resize_token_embeddings(len(tokenizer))

  questions = quiz_questions
  # answers = [0 for i in range(0, questions)]

  count0 = 0.0
  count1 = 0.0
  count2 = 0.0
  count3 = 0.0
  count4 = 0.0
  total = 0.0
  model.eval()
  with torch.no_grad():        
    for i, choices in enumerate(questions):
        #print(choices)
        # print(i)
        # if i % 500 == 0 and i != 0:
        #   print(f"On iteration {i}, accuracy is {count0/total} count1: {count1/total} count2: {count2/total} count3: {count3/total} count4: {count4/total}")
        encoded_choices = [tokenizer.encode(s) for s in choices]
        cls_token_location = [tokens.index(tokenizer.cls_token_id) for tokens in encoded_choices]
        #check if some choices are longer than the others
        max_len = max([len(tokens) for tokens in encoded_choices])
        check = True
        for tokens in encoded_choices:
            if len(tokens) < max_len:
                check = False
        if check:
            input_ids = torch.tensor(encoded_choices).unsqueeze(0)  # Batch size: 1, number of choices: 2
            if len(input_ids) >= 1024:
              continue
            mc_token_ids = torch.tensor([cls_token_location])  # Batch size: 1
            input_ids = input_ids.to(device)
            mc_token_ids = mc_token_ids.to(device)
            outputs = model(input_ids, mc_token_ids=mc_token_ids)
            lm_logits = outputs.logits
            mc_logits = outputs.mc_logits
            if mc_logits.argmax().item() == 0:
                count0 += 1
            elif mc_logits.argmax().item() == 1:
                count1 += 1
            elif mc_logits.argmax().item() == 2:
                count2 += 1
            elif mc_logits.argmax().item() == 3:
                count3 += 1
            elif mc_logits.argmax().item() == 4:
                count4 += 1
            
            total += 1
  print(f"FINAL count0: {count0/total} count1: {count1/total} count2: {count2/total} count3: {count3/total} count4: {count4/total}")
  return count0/total
  # print(len(questions))
  # print(count0)
  # print(count1)
  # print(count2)
  # print(count3)
  # print(count4)
  # print(total)